In [2]:
'''
this program parses the description file,
in order to map all the string of categorical attributes to an integer
so that they can be treated as numerical attribute
and then removes the columns which have more than 100 nulls 
'''

'\nthis program parses the description file,\nin order to map all the string of categorical attributes to an integer\nso that they can be treated as numerical attribute\nand then removes the columns which have more than 100 nulls \n'

In [1]:
import pandas
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:

'''
@params:
    filename: a filename of the description.txt from kaggle competition describing each columns of dataset
@returns:
    dictionary of mapping for each string type of categorical values for each column with an integer
'''
def buildJSON(filename):
    with open(filename,'r') as f:
        d = dict()
        temp = ''
        count = 0
        for l in f:
            if l.strip() == '':continue
            if ':' in l:
                temp = l.split(':')[0]
                d[temp] = dict()
                count = 0
            else:
                lst = l.strip().split('\t')
                try:
                    int(lst[0])
                except ValueError:
                    d[temp][lst[0]] = count
                count += 1
    return d
#print(d)

In [3]:
'''
@params
    df: a dataframe from which the null columns are to be removied
    threshold: the total count of nulls in the column to decide whether to remove that column or not
@returns:
    dataframe with deleted columns based on the threshold from the provided dataframe 'df'
'''
def removeNullCols(df,threshold):
    count_null = sum(pandas.isnull(df[df.columns]),axis=0)
    return df[count_null[count_null < threshold].index]

In [4]:
filename = 'dataset/data_description.txt'
d = buildJSON(filename)
df = pandas.read_csv('dataset/train.csv')
processed_df = removeNullCols(df,50)
processed_df = processed_df.replace(d)
#print(d)

In [5]:

from sklearn.ensemble import RandomForestRegressor
processed_df = processed_df.fillna(processed_df.median(axis=0))
y = processed_df['SalePrice']
X = processed_df.drop('SalePrice',axis=1)
cols = X.columns

C:\Users\chitr\AppData\Local\conda\conda\envs\cse627\lib\site-packages\sklearn\utils\__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
C:\Users\chitr\AppData\Local\conda\conda\envs\cse627\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
from sklearn.ensemble import RandomForestRegressor
processed_df = processed_df.fillna(processed_df.median(axis=0))
y = processed_df['SalePrice']
X = processed_df.drop('SalePrice',axis=1)
cols = X.columns
#sum(pandas.isna(processed_df))

In [28]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error as mse
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)

tdf = pandas.get_dummies(df)
tdf = tdf.fillna(tdf.median(axis=0))
ty = tdf['SalePrice']
tX = tdf.drop('SalePrice',axis=1)
for train_index, test_index in skf.split(tX, ty):
    X_train, X_test = tX.iloc[train_index], tX.iloc[test_index]
    y_train, y_test = ty.iloc[train_index], ty.iloc[test_index]
    print(X_train.shape)
    print(len(X_train),len(X_test))
    r1 = RandomForestRegressor(50)
    r1.fit(X_train,y_train)
    ty_test = r1.predict(X_test)
    print(mse(y_test,ty_test))
    print(len(ty_test),'predicted array\'s length')
    
#sum(pandas.isna(X[X.columns]))

C:\Users\chitr\AppData\Local\conda\conda\envs\cse627\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


(728, 289)
728 732
2592427878.8825135
732 predicted array's length
(1156, 289)
1156 304
421100526.2524487
304 predicted array's length
(1258, 289)
1258 202
400033097.2775504
202 predicted array's length
(1329, 289)
1329 131
870154386.6345707
131 predicted array's length
(1369, 289)
1369 91
439729766.1415779
91 predicted array's length


In [29]:
skf1 = StratifiedKFold(n_splits=10)
for train_index,test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index],X.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    print(X_train.shape)
    r = RandomForestRegressor(100)
    r.fit(X_train,y_train)
    y_pred = r.predict(X_test)
    print(mse(y_test,y_pred))

C:\Users\chitr\AppData\Local\conda\conda\envs\cse627\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


(728, 69)
2519850091.4385247
(1156, 69)
399204821.4493043
(1258, 69)
394386821.6488178
(1329, 69)
1262237382.7285686
(1369, 69)
426236135.7211549


In [7]:
test_df = pandas.read_csv('dataset/test.csv')
rf = RandomForestRegressor(100)
rf.fit(X,y)
test_df = test_df[cols]
test_df = test_df.replace(d)
test_df = test_df.fillna(test_df.median(axis=0))
y_pred = rf.predict(test_df)

In [38]:
submission = pandas.concat([test_df['Id'],pandas.Series(y_pred,name='SalePrice')],axis=1)
submission.to_csv('dataset/submission_1.csv',index=False)
#submission.shape

In [9]:
from sklearn.ensemble import GradientBoostingRegressor as GBR
gbr = GBR()
gbr.fit(X,y)
y_pred = (gbr.predict(test_df)+rf.predict(test_df))/2

In [33]:
submission = pandas.concat([test_df['Id'],pandas.Series(y_pred,name='SalePrice')],axis=1)
submission.to_csv('dataset/submission_3.csv',index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
    "loss":["ls","lad"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[100,150,200]
    }
reg = GridSearchCV(GBR(), parameters, cv=10, n_jobs=-1)
reg.fit(X,y)
y_pred = reg.predict(test_df)

In [ ]:
submission = pandas.concat([test_df['Id'],pandas.Series(y_pred,name='SalePrice')],axis=1)
submission.to_csv('dataset/submission_4.csv',index=False)